In [4]:
#!/usr/bin/env python
# coding: utf-8

# Import required libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm

In [5]:

# Define a threshold for matching errors
space_match_threshold = 1 * u.arcsec
MJD_tolerance = 0.00034 #30 sec in units of day 

# Define file paths.
sum_path = {}
sum_path["star"] = '../truth_star/truth_star_summary_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_star/truth_star_summary_v1-0-0.parquet'
sum_path["sn"] = "../truth_sn/truth_sn_summary_v1-0-0.parquet" 
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_sn/truth_sn_summary_v1-0-0.parquet'

var_path = {}
var_path["star"] = '../truth_star/truth_star_variability_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_star/truth_star_variability_v1-0-0.parquet'
var_path["sn"] = '../truth_sn/truth_sn_variability_v1-0-0.parquet'
#'/sdf/data/rubin/shared/dc2_run2.2i_truth/truth_sn/truth_sn_variability_v1-0-0.parquet'

detection_csv_pth = "../sources_with_labels.csv" #'exported_sources.csv'

#get DIA detections
dia_detections = pd.read_csv(detection_csv_pth, index_col="diaSourceId") #formerly known as exported_csv
#only for now based on the csv we are reading
dia_detections.drop(["real"], axis=1, inplace=True)    

dia_detections

,ra,dec,midpointMjdTai,type
diaSourceId,,,,
1257927201521665,55.760339,-32.260622,59583.125051,NaN
1257927201521666,55.674078,-32.283857,59583.125051,NaN
1257927201521667,55.552914,-32.306395,59583.125051,NaN
1257927201521668,55.547689,-32.309278,59583.125051,NaN
1257927201521669,55.570127,-32.306400,59583.125051,NaN
...,...,...,...,...
660667525163384915,55.889519,-32.485637,61392.194195,star
661047079476396040,55.863218,-32.167598,61393.204087,star
662500331589992590,55.971559,-32.358853,61404.195949,star


## load truth catalogs

In [6]:
# Get mind and max ra and dec values to filter out unnecessary records.
max_exp_ra, min_exp_ra = dia_detections.ra.max(), dia_detections.ra.min()
max_exp_dec, min_exp_dec = dia_detections.dec.max(), dia_detections.dec.min()


catalog = {}
result_sum = {}

# Stage 1: Match sources in Space.
for s in ["star", "sn"]:
    # Read Parquet and CSV files to begin ground truth derivation.
    result_sum[s] = pd.read_parquet(sum_path[s])

    # Keep only those records from summary tables which are within the max ra and dec values in the exported sources.
    result_sum[s] = result_sum[s][(result_sum[s]['ra'] >= min_exp_ra) & (result_sum[s]['ra'] <= max_exp_ra) &\
                                    (result_sum[s]['dec'] >= min_exp_dec) & (result_sum[s]['dec'] <= max_exp_dec)]


    # Initialize astropy.coordinates.SkyCoord class for matching in space.
    catalog[s] = SkyCoord(ra=result_sum[s].ra, dec=result_sum[s].dec, unit=u.deg)

# Match exported sources with stars and supernovae.
detections_cat = SkyCoord(ra=dia_detections.ra, dec=dia_detections.dec, unit=u.deg)

In [23]:
# By default, set on_source = 0 and real=0 (bogus) for all values in the exported sources.
dia_detections['on_source'] = 0
dia_detections['real'] = 0
dia_detections['type'] = None

## spatial crossmatch

In [10]:
star_idx, star_d2d, star_d3d = detections_cat.match_to_catalog_sky(catalog['star'])
sn_idx, sn_d2d, sn_d3d = detections_cat.match_to_catalog_sky(catalog['sn'])

star_mask = star_d2d < space_match_threshold #remove matches that are too far
sn_mask = sn_d2d < space_match_threshold #remove matches that are too far

print(f"{np.sum(star_mask)} of {len(detections_cat)} stars matched before applying threshold")
print(f"{np.sum(sn_mask)} of {len(detections_cat)} sne matched before applying threshold")

7155 of 25446 stars matched before applying threshold
146 of 25446 sne matched before applying threshold


In [27]:


# Get all matched stars
matched_star_idx = star_idx[star_mask] #index in stars of matched dia_detections
print(f"Number of matched stars in Stage #1: {len(matched_star_idx)}")

# Get all matched supernovae
matched_sn_idx = sn_idx[sn_mask] #index in sn_cat of matched dia_detections
print(f"Number of matched sne in Stage #1: {len(matched_sn_idx)}")

dia_idx_stars = dia_detections.index[star_mask]
dia_idx_sn = dia_detections.index[sn_mask]

# Assign the variability sources catalog id to the detections
dia_detections["id"] = None
dia_detections.loc[dia_idx_stars,"id"] = result_sum["star"].iloc[matched_star_idx]["id"].to_numpy()
dia_detections.loc[dia_idx_sn,"id"] = result_sum['sn'].iloc[matched_sn_idx]["id"].to_numpy()



Number of matched stars in Stage #1: 7155
Number of matched sne in Stage #1: 146


In [28]:
dia_detections.head()

,ra,dec,midpointMjdTai,type,id,on_source,real
diaSourceId,,,,,,,
1257927201521665,55.760339,-32.260622,59583.125051,None,30321355720,0,0
1257927201521666,55.674078,-32.283857,59583.125051,None,None,0,0
1257927201521667,55.552914,-32.306395,59583.125051,None,None,0,0
1257927201521668,55.547689,-32.309278,59583.125051,None,None,0,0
1257927201521669,55.570127,-32.306400,59583.125051,None,None,0,0


In [29]:
# The spatially matched detections get on_source = 1

dia_detections.loc[dia_idx_sn, "on_source"] = 1
dia_detections.loc[dia_idx_stars , "on_source"] = 1
dia_detections.loc[dia_idx_sn, "type"] = "sn"
dia_detections.loc[dia_idx_stars, "type"] = "star"
dia_detections

,ra,dec,midpointMjdTai,type,id,on_source,real
diaSourceId,,,,,,,
1257927201521665,55.760339,-32.260622,59583.125051,star,30321355720,1,0
1257927201521666,55.674078,-32.283857,59583.125051,None,None,0,0
1257927201521667,55.552914,-32.306395,59583.125051,None,None,0,0
1257927201521668,55.547689,-32.309278,59583.125051,None,None,0,0
1257927201521669,55.570127,-32.306400,59583.125051,None,None,0,0
...,...,...,...,...,...,...,...
660667525163384915,55.889519,-32.485637,61392.194195,star,31411443281,1,0
661047079476396040,55.863218,-32.167598,61393.204087,star,31102009372,1,0
662500331589992590,55.971559,-32.358853,61404.195949,star,31405685742,1,0


In [30]:
# Print a summary at the end of first round of matching.
print("Summary at the end of First Stage:")
print(f"detections on a source", dia_detections["on_source"].sum(), "\n")
print(f"class detection: {dia_detections.groupby('type').count().iloc[:,0]}")

Summary at the end of First Stage:
detections on a source 7301 

class detection: type
sn       146
star    7155
Name: ra, dtype: int64


In [32]:
# Stage 2: Match sources in time.

matched_index = dia_detections["on_source"] > 0
matched = {}
matched["sn"] = dia_detections.loc[dia_idx_sn]
matched["star"] = dia_detections.loc[dia_idx_stars]


for s in ["sn", "star"]:
    print(f"working on class: {s}")
    
    # Get a list of all the unique MJDs of sources that matched in the previous stage for the object type.
    mjd_matched_in_space = matched[s].midpointMjdTai.unique()

    # Get min and max MJD values required for matching.
    max_mjd, min_mjd = mjd_matched_in_space.max(), mjd_matched_in_space.min()

    # Read star/sn lightcurve variability parquet for the object type
    df_var = pd.read_parquet(var_path[s])
    
    # Filter out records with unwanted MJDs.
    df_var = df_var[(df_var.MJD >= min_mjd) & (df_var.MJD <= max_mjd)]
    print(f"need to examine {len(df_var)} variability entries")
    
    for detected in tqdm(matched[s].index): #loop over indices of on_source detection 
        mask_matching_ids = df_var.id == matched[s].loc[detected].id #mask for sources with matching id in variability file 
        if np.any(np.abs(df_var[mask_matching_ids].MJD - matched[s].loc[detected].midpointMjdTai) <= MJD_tolerance):
            dia_detections.loc[detected, "real"] = 1
            print(detected)
    del df_var

dia_detections[dia_detections.real == 1]

working on class: sn
need to examine 12757691 variability entries


 12%|█████                                    | 18/146 [00:00<00:00, 177.33it/s]

342457652904722669


 40%|████████████████▎                        | 58/146 [00:00<00:00, 193.00it/s]

351362700657295472
351363319669456940


...

645816020564443153


100%|██████████████████████████████████████▉| 7140/7155 [12:46<00:01,  9.04it/s]

647699403989058266


100%|███████████████████████████████████████| 7155/7155 [12:48<00:00,  9.31it/s]


,ra,dec,midpointMjdTai,type,id,on_source,real
diaSourceId,,,,,,,
104057445445271766,55.810521,-32.439969,59840.233977,star,31411437889,1,1
104057445445271773,55.685438,-32.420221,59840.233977,star,31107745884,1,1
104057445445271824,55.838160,-32.416255,59840.233977,star,31405691981,1,1
104057445445271838,55.879441,-32.413054,59840.233977,star,31411435379,1,1
104057445445271862,55.816092,-32.372750,59840.233977,star,31107745011,1,1
...,...,...,...,...,...,...,...
510734470488260694,55.966513,-32.222557,60973.167688,star,30321355633,1,1
554641942630105130,55.792227,-32.292353,61101.060926,star,30830343259,1,1
627179613616865357,55.831304,-32.192921,61297.243962,star,31405665055,1,1
